In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
%matplotlib inline
import math
from keras.utils import np_utils

In [ ]:
from tensorflow.keras.datasets import cifar10

(train_x, train_y), (test_x, test_y) = cifar10.load_data()

num_classes = 10
train_y_cls = np_utils.to_categorical(train_y,num_classes)
test_y_cls = np_utils.to_categorical(test_y,num_classes)

print("Size of:")
print("- Training-set:\t\t{}".format(len(train_x)))
print("- Test-set:\t\t{}".format(len(test_x)))

In [ ]:
def load_label_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

class_names = load_label_names()

def label_to_array(cls_true):
    true = cls_true.tolist()
    true_label = []
    for l in true:
        true_label += l
    return true_label
    

In [ ]:
#Data Dimensions

# CIFAR-10 images are 32 pixels in each dimension.
img_size = 32

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images:
num_channels = 3

# Number of classes, one class for each of 10 digits.
num_classes = 10

In [ ]:
#Plotting Images

def plot_images(images, cls_true, cls_pred=None, smooth=True):

    # Create figure with sub-plots.
    fig, axes = plt.subplots(3, 3)

    # Adjust vertical spacing if we need to print ensemble and best-net.
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Interpolation type.
        if smooth:
            interpolation = 'spline16'
        else:
            interpolation = 'nearest'

        # Plot image.
        ax.imshow(images[i, :, :, :],
                  interpolation=interpolation)
            
        # Name of the true class.
        cls_true_name = class_names[cls_true[i]]

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true_name)
        else:
            # Name of the predicted class.
            cls_pred_name = class_names[cls_pred[i]]

            xlabel = "True: {0}\nPred: {1}".format(cls_true_name, cls_pred_name)

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
# Get the first 9 images from the test-set.
images = train_x[0:9]

# Get the true classes for those images.
cls_true = train_y[0:9]
cls_true = label_to_array(cls_true)

# Plot the images and labels using our helper-function above.
plot_images(images, cls_true)


In [ ]:
# Normalise Data
train_x = (train_x.astype('float32'))/255.
test_x = (test_x.astype('float32'))/255.

In [6]:
# Split data into train and validation sets
from sklearn.model_selection import train_test_split


x_train, x_validation, y_train, y_validation = train_test_split(train_x, train_y_cls,
                                                                test_size=0.2, random_state=42)

NameError: name 'train_x' is not defined

###  Sequential model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam

In [ ]:
def alexnet_sequential(width, height, channels, classes):
    # Initialize the model along with the input shape to be
    # "channels last" ordering
    # Note: in Alexnet, all paddings were 'valid'
    # Here, we are taking the size of all filters as 3x3 and adding BatchNormalisation after each layer
    
    model = Sequential()
    inputShape = (height, width, channels)
    
    # Define the first CONV Layer
    model.add(Conv2D(48, (3,3), # In Alexnet this kernel size is 11
                     input_shape=inputShape,
                     name='conv_layer1',
                     dtype=tf.float32,
                     strides=1, ## Originally: 4
                     padding='same')) ## Orignally:'valid'
    model.add(Activation("relu", name='activation1'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2,
                          padding="same", name='maxpool1'))
    model.add(BatchNormalization(name='BatchNorm1'))
    
    # Define the second CONV Layer
    model.add(Conv2D(96, (3, 3),
                     name='conv_layer2',
                     dtype=tf.float32,
                     padding='same')) 
    model.add(Activation("relu", name='activation2'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2,
                          padding='same', name='maxpool2'))
    model.add(BatchNormalization(name='BatchNorm2'))
    
    # Define the third CONV Layer
    model.add(Conv2D(192, (3, 3), padding='same',
                     name='conv_layer3',
                     dtype=tf.float32))
    model.add(Activation("relu", name='activation3'))
    model.add(BatchNormalization(name='BatchNorm3'))
    
    # Define the fourth CONV Layer
    model.add(Conv2D(192, (3, 3), padding='same',
                     name='conv_layer4',
                     dtype=tf.float32))
    model.add(Activation("relu", name='activation4'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2,
                          padding='same', name='maxpool3'))
    
    model.add(BatchNormalization(name='BatchNorm4'))
    
    ## Define fifth CONV Layer
    model.add(Conv2D(256, (3, 3), padding='same',
                     name='conv_layer5',
                     dtype=tf.float32))
    model.add(Activation("relu", name='activation5'))
    model.add(MaxPooling2D(pool_size = 2, strides = 2,
                          padding='same', name='maxpool4'))
    model.add(BatchNormalization(name='BatchNorm5'))
    
    ## Note: Only make 2/3 Conv layers as more will
    ##       require a lot of computations
    
    # First Dense Layer with ReLU activation
    model.add(Flatten())
    
    # Second Dense Layer with ReLU activation
    model.add(Dense(512, name='dense1'))
    model.add(Activation("relu", name='activation6'))
    model.add(Dropout(0.4))

    # Third Dense Layer with ReLU activation
    model.add(Dense(256, name='dense2'))
    model.add(Activation("relu", name='activation7'))
    model.add(Dropout(0.4))

    # Final Dense Layer with softmax activation
    model.add(Dense(num_classes, name='dense3'))
    model.add(Activation("softmax", name='activation8'))
    
    # return the constructed network architecture
    return model


In [ ]:
## Create Model
model = alexnet_sequential(32, 32, 3, num_classes)

In [ ]:
num_epochs = 2

# Initialize the optimizer and compile the model
optimizer = Adam(lr=0.01, decay=0.0005 / num_epochs)
print("Training network...")
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
             metrics=['accuracy'])


### Data Augmnetation

In [ ]:
# Use Data Augmentation or not
data_augmentation = True

In [ ]:
# Run training, with or without data augmentation.
if not augment_data:
    print('Not using data augmentation.')
    h = model.fit(x_train,
                  y_train,
                  batch_size = 128,
                  validation_data=(x_validation,y_validation),
                  epochs = num_epochs,
                  verbose = 1)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
                        epochs=num_epochs, verbose=1, workers=4, 
                        validation_data=(x_validation,y_validation))

In [ ]:
model.summary()

In [ ]:
#Evaluate on Test Set
result = model.evaluate(test_x, test_y_cls)

In [ ]:
#Predict on test_set
pred_y = model.predict(test_x)
pred_cls = np.argmax(pred_y, axis=1)


In [ ]:
#Some predicted Images
# Get the first 9 images from the test-set.
images = test_x[0:9]

# Get the true classes for those images.
cls_true = test_y[0:9]
cls_true = label_to_array(cls_true)

#Get Predicted Classes
cls_pred = pred_cls[0:9]
cls_pred = cls_pred.tolist()

# Plot the images and labels using our helper-function above.
plot_images(images, cls_true, cls_pred)